<a href="https://colab.research.google.com/github/Maspie/Customer-Life-Time-Value-Prediction-and-Clustering/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/My Drive/Clustered Dataset/cleaned_cltv.csv')

url = "https://drive.google.com/uc?export=download&id=176F7rnRdMAW9OPIDrdDpXiQfnUZ3TXv0"
dataset_cluster = pd.read_csv(url)


Previewing the dataset before dropping features in the main EDA notebook.

In [ ]:
dataset_cluster.head()

Applying Minibatch Kmeans since size of dataset is big and finding number of clustering through elbow method.

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler

import numpy as np
scaler = MinMaxScaler()

dataset_cluster['cltv'] = scaler.fit_transform(dataset_cluster[['cltv']])
dataset_cluster['claim_amount'] = scaler.fit_transform(dataset_cluster[['claim_amount']])
dataa = dataset_cluster.copy()
# Determine the optimal number of clusters using the elbow method
sse = []
k_candidates = range(1, 15)

for k in k_candidates:
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=100, random_state=42)
    kmeans.fit(dataset_cluster)
    sse.append(kmeans.inertia_)

# Plot SSE (elbow method)
plt.figure(figsize=(10, 6))
plt.plot(k_candidates, sse, '-o')
plt.xlabel('Number of clusters, k')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method for Optimal k')
plt.show()

In [ ]:
dataa.head()

Mini Batch Kmeans, SSE and Davies

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import davies_bouldin_score

n_clusters = 3
batch_size = 1000

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, random_state=100)

kmeans.fit(dataset_cluster)

centers_kmeans = kmeans.cluster_centers_
labels_kmeans = kmeans.labels_

if not isinstance(dataset_cluster, pd.DataFrame):
    df_kmeans = pd.DataFrame(dataset_cluster)
else:
    df_kmeans = dataset_cluster.copy()

df_kmeans['Cluster'] = labels_kmeans

# Calculate SSE for each cluster
sse_per_cluster = np.zeros(n_clusters)
for i in range(n_clusters):
    cluster_points = df_kmeans[df_kmeans['Cluster'] == i].drop('Cluster', axis=1)
    distances = np.linalg.norm(cluster_points - centers_kmeans[i], axis=1)
    sse_per_cluster[i] = np.mean(np.square(distances))

# Mean SSE across all clusters
mean_sse = np.mean(sse_per_cluster)

print(f"Mean SSE per cluster: {sse_per_cluster}")
print(f"Mean SSE across all clusters: {mean_sse}")

db_score_kmeans = davies_bouldin_score(df_kmeans.drop('Cluster', axis=1), labels_kmeans)
print(f"Davies-Bouldin Index: {db_score_kmeans}")

sse_kmeans = kmeans.inertia_
print(f"Sum of Squared Error (Total Inertia): {sse_kmeans/1017209}")


BIRCH , SSE and Davies Score



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import Birch
from sklearn.metrics import davies_bouldin_score

# dataa = pd.DataFrame(np.random.rand(100, 2))

if isinstance(dataa, pd.DataFrame):
    dataa = dataa.values

# Setting up the BIRCH clustering
birch = Birch(n_clusters=3, threshold=0.6, branching_factor=20)
labels_birch = birch.fit_predict(dataa)

# Compute the centroids of each cluster
centroids_birch = np.array([dataa[labels_birch == i].mean(axis=0) for i in range(3)])

sse_per_cluster = np.zeros(3)

# Calculate the SSE for each cluster
for i in range(3):
    cluster_points = dataa[labels_birch == i]
    cluster_center = centroids_birch[i]
    sse = np.sum((cluster_points - cluster_center) ** 2)
    sse_per_cluster[i] = sse

# Calculate mean SSE by dividing by the number of points in each cluster
num_points_per_cluster = np.array([sum(labels_birch == i) for i in range(3)])
mean_sse_per_cluster = sse_per_cluster / num_points_per_cluster

# Print the mean SSE for each cluster
for i, mean_sse in enumerate(mean_sse_per_cluster):
    print(f"Mean SSE for Cluster {i}: {mean_sse}")

# Calculate and print the overall SSE divided by total data points
total_sse = np.sum(sse_per_cluster)
print(f"Overall Mean SSE (divided by total data points): {total_sse / len(dataa)}")

# Calculate and print the Davies-Bouldin Score
db_score = davies_bouldin_score(dataa, labels_birch)
print(f"Davies-Bouldin Index: {db_score}")

# Mean SSE across all clusters divided by the total number of data points
mean_sse_across_clusters = total_sse / len(dataa)
print(f"Mean SSE across all clusters (divided by total data points): {mean_sse_across_clusters}")



In [ ]:
print(sse_per_cluster/len(dataa))

2D RERESENTATION, PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions to 2 for visualization
pca = PCA(n_components=2)
dataa_pca = pca.fit_transform(dataset_cluster)

# Sample 100 points from each cluster
samples_per_cluster = 100
colors = ['red', 'blue', 'green']  # Colors for clusters 0, 1, 2

def sample_data(labels):
    sampled_indices = np.hstack([np.random.choice(np.where(labels == i)[0], samples_per_cluster, replace=False) for i in range(3)])
    return dataa_pca[sampled_indices], labels[sampled_indices]

# Sample data for Birch and KMeans
dataa_pca_birch, labels_birch_sampled = sample_data(labels_birch)
dataa_pca_kmeans, labels_kmeans_sampled = sample_data(labels_kmeans)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Plotting function for 2D
def plot_clusters(ax, data, labels, centroids, title, score_text):
    for i, color in enumerate(colors):
        cluster_data = data[labels == i]

        ax.scatter(cluster_data[:, 0], cluster_data[:, 1], c=color, label=f'Cluster {i}', alpha=0.5)


        centroid = centroids[i]
        ax.scatter(centroid[0], centroid[1], marker='x', s=100, color='k')

        # lines from samples to the centroid
        for point in cluster_data:
            ax.plot([point[0], centroid[0]], [point[1], centroid[1]], 'k--', linewidth=0.5, alpha=0.3)

    ax.set_title(title)
    ax.set_xlabel('PCA Component 1')
    ax.set_ylabel('PCA Component 2')
    plt.legend(loc='upper right')
    ax.text(0.05, -0.1, score_text, transform=ax.transAxes)

# Calculate centroids for the samples in 2D
centroids_birch_pca = np.array([dataa_pca_birch[labels_birch_sampled == i].mean(axis=0) for i in range(3)])
centroids_kmeans_pca = np.array([dataa_pca_kmeans[labels_kmeans_sampled == i].mean(axis=0) for i in range(3)])

# Plot for Birch
plot_clusters(ax1, dataa_pca_birch, labels_birch_sampled, centroids_birch_pca, 'Birch Clustering', 'SSE: {:.2f}\nDavies-Bouldin Index: {:.2f}'.format(total_sse, db_score))

# Plot for KMeans
plot_clusters(ax2, dataa_pca_kmeans, labels_kmeans_sampled, centroids_kmeans_pca, 'KMeans Clustering', 'SSE: {:.2f}\nDavies-Bouldin Index: {:.2f}'.format(sse_kmeans, db_score_kmeans))

plt.show()



3D RERESENTATION, PCA


In [ ]:

from mpl_toolkits.mplot3d import Axes3D

# Taking 100 samples per cluster
samples_per_cluster = 100
colors = ['red', 'blue', 'green']

# Sample indices for each cluster
def sample_indices(labels, n_samples_per_cluster):
    unique_labels = np.unique(labels)
    indices = np.hstack([np.random.choice(np.where(labels == label)[0], n_samples_per_cluster, replace=False)
                         for label in unique_labels])
    return indices

# Sample indices from the original labels
birch_indices = sample_indices(labels_birch, samples_per_cluster)
kmeans_indices = sample_indices(labels_kmeans, samples_per_cluster)

# Applying PCA to reduce dimensions to 3 for visualization
pca = PCA(n_components=3)

# Dataframe check
if isinstance(dataset_cluster, pd.DataFrame):
    dataa_pca_3d_birch = pca.fit_transform(dataset_cluster.iloc[birch_indices].values)
    dataa_pca_3d_kmeans = pca.transform(dataset_cluster.iloc[kmeans_indices].values)
else:
    dataa_pca_3d_birch = pca.fit_transform(dataset_cluster[birch_indices])
    dataa_pca_3d_kmeans = pca.transform(dataset_cluster[kmeans_indices])

# Labels update
labels_birch_sampled = labels_birch[birch_indices]
labels_kmeans_sampled = labels_kmeans[kmeans_indices]

# Centroid calculation
centroids_birch = np.array([dataa_pca_3d_birch[labels_birch_sampled == i].mean(axis=0) for i in np.unique(labels_birch_sampled)])
centroids_kmeans = np.array([dataa_pca_3d_kmeans[labels_kmeans_sampled == i].mean(axis=0) for i in np.unique(labels_kmeans_sampled)])

# Visualization in 3D
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(121, projection='3d', title='Birch Clustering 3D')
ax2 = fig.add_subplot(122, projection='3d', title='KMeans Clustering 3D')

# plot clusters and centroid lines in 3D
def plot_clusters_3d(ax, data, labels, centroids, title):
    for i, color in enumerate(colors):

        cluster_data = data[labels == i]

        ax.scatter(cluster_data[:, 0], cluster_data[:, 1], cluster_data[:, 2], c=color, label=f'Cluster {i}', alpha=0.5)

        ax.scatter(centroids[i][0], centroids[i][1], centroids[i][2], c='k', marker='x', s=100)

        # lines from each point to the centroid
        for point in cluster_data:
            ax.plot([point[0], centroids[i][0]], [point[1], centroids[i][1]], [point[2], centroids[i][2]], 'k-', linewidth=0.5, alpha=0.5)

    ax.set_title(title)
    ax.set_xlabel('PCA Component 1')
    ax.set_ylabel('PCA Component 2')
    ax.set_zlabel('PCA Component 3')
    ax.legend()

# Plot for Birch in 3D
plot_clusters_3d(ax1, dataa_pca_3d_birch, labels_birch_sampled, centroids_birch, 'Birch Clustering 3D')

# Plot for KMeans in 3D
plot_clusters_3d(ax2, dataa_pca_3d_kmeans, labels_kmeans_sampled, centroids_kmeans, 'KMeans Clustering 3D')

plt.show()
